In [1]:
!pip install -q tritonclient[http]
!pip install -q pandas scikit-learn

In [2]:
import tritonclient.http as httpclient
from tritonclient.utils import np_to_triton_dtype

In [3]:
import pandas as pd
import numpy as np
import json

In [10]:
client = httpclient.InferenceServerClient(url='localhost:8000')

In [46]:
data_infer2 = pd.read_csv("data_infer.csv").iloc[:5].astype(str)

In [19]:
data_infer = pd.read_csv("data_infer.csv").iloc[:5]

In [20]:
data_infer.head()

,User,Card,Year,Month,Day,Time,Amount,Use Chip,Merchant Name,Merchant City,Merchant State,Zip,MCC,Errors?
0,1005,0,2013,6,5,22:03,$-180.00,Swipe Transaction,7834055923142137930,Orlando,FL,32808.0,3395,NaN
1,134,5,2015,2,16,18:34,$99.10,Chip Transaction,-1548923525906069124,Tyler,TX,75706.0,4900,NaN
2,1270,0,2009,11,27,18:44,$82.42,Swipe Transaction,-7329090674102508598,Saint Charles,IL,60174.0,5812,NaN
3,109,0,2010,11,3,16:21,$0.45,Swipe Transaction,-6571010470072147219,Dyer,IN,46311.0,5499,NaN
4,1853,2,2006,3,27,12:25,$11.41,Swipe Transaction,6135208568923449408,East Peoria,IL,61611.0,9402,NaN


In [24]:
STR_COL_NAMES = ['Time', 'Amount', 'Zip', 'MCC', 'Merchant Name', 'Use Chip', 'Merchant City', 'Merchant State', 'Errors?']

In [31]:
def prepare_tensor(name, input_data):
    tensor = httpclient.InferInput(
        name, input_data.shape, np_to_triton_dtype(input_data.dtype))
    tensor.set_data_from_numpy(input_data)
    return tensor

In [67]:
data_dict = {}
inputs = []
for col in data_infer.columns:
    if col in STR_COL_NAMES:
        data_dict[col] = np.expand_dims(data_infer[col].astype(str).values, axis=-1)
    else:
        data_dict[col] = np.expand_dims(data_infer[col].astype(np.float32).values, axis=-1)
    inputs.append(prepare_tensor(col, data_dict[col]))

In [68]:
data_dict['User'].tolist()

[[1005.0], [134.0], [1270.0], [109.0], [1853.0]]

In [69]:
model_name = "ensemble"

In [70]:
response = client.infer(model_name,
                        inputs)

In [71]:
output_data = response.as_numpy("predictions")

In [73]:
output_data

(5,)

In [28]:
output_data.astype(str)

array(['NOT FRAUD', 'NOT FRAUD', 'NOT FRAUD', 'NOT FRAUD'], dtype='<U9')